In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
print(tf.__version__)

2.15.0


In [7]:
from tensorflow.keras.models import load_model
from keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

# Path to your HDF5 file
model_path = 'models/speed/lane_navigation_check.h5'

# Load the model
speed_model = load_model(model_path)

# Now the model is ready to use
speed_model.summary()

Model: "Nvidia_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [8]:


# Path to your HDF5 file
weights_path = 'models/angle/angle_navigation.h5'

def nvidia_model():
    model = Sequential(name='Nvidia_A_Model')
    model.add(tf.keras.applications.mobilenet_v2.MobileNetV2(include_top = False, weights="imagenet", input_shape=(200, 200, 3)))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(Dense(1, activation = 'sigmoid'))
    model.layers[0].trainable = False

    model.compile(optimizer=tf.optimizers.RMSprop(lr=0.01), loss='binary_crossentropy', metrics='accuracy')

    return model


# Create the model
angle_model = nvidia_model()

# Load the weights
angle_model.load_weights(weights_path)

In [2]:
import sys
import os

# # Construct the absolute path to the module's directory
# # os.path.abspath returns the absolute path of the given path
# # os.path.join combines paths in a way that's safe for any operating system
# module_dir = os.path.abspath(os.path.join('models', 'maggie'))
# print(module_dir)

# # Add the module's directory to sys.path
# if module_dir not in sys.path:
#     sys.path.append(module_dir)

# Now you can import the Model class
from model import Model

# Use the Model class as needed
model_instance = Model()


In [7]:
# FUNCTION TO READ TESTING DATA
def read_dataset_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.cast(img, tf.float32) 
    # print(img.shape)
    # img = tf.image.resize(img,[224,224])
    # img = tf.reshape(img,[1,224,224,3])
    return img

def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV) / 255.0
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,200)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    return image

In [9]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,200)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    return image

In [20]:
def predict(image):
    # angles = np.arange(17)*5+50
    image = img_preprocess(image)
    image_reshaped = image.reshape((1, 200, 200, 3))
    # print(image_reshaped.shape)
    pred_speed = speed_model.predict(image_reshaped)[0]
    # print(pred_speed)
    # speed = pred_speed[0].astype(int)*35
    pred_angle = angle_model.predict(image_reshaped)[0]
    # angle = angles[np.argmax(pred_angle)]
    # print('angle:', angle,'speed:', speed)
    
    return pred_angle, pred_speed

In [29]:
# TO CREATE PREDICTIONS OF THE TEST DATA
predictions = []
test_dir = os.path.abspath(os.path.join('test_data','test_data'))

for i in range(1,1021):
    img_path = f"{test_dir}/{i}.png"
    img = my_imread(img_path)
    # print(img.shape)
    # angle_pred, speed_pred = model_instance.predict(img)
    
    angle_pred, speed_pred = predict(img)


    if abs(1-speed_pred[0]) < abs(speed_pred[0]-0):
            speed_pred[0] = 1
    else:
        speed_pred[0] = 0


    predictions.append([angle_pred[0], speed_pred[0]])
    # angle_pred = angle
    print([(angle_pred[0]*80)+50, speed_pred[0]*35])
    # predictions.append([(angle_pred-50)/80, (speed_pred-0)/35])
    


1/1 [==============================] - 0s 65ms/step
[90.60556411743164, 0.0]
1/1 [==============================] - 0s 36ms/step
[105.59391498565674, 35.0]
1/1 [==============================] - 0s 32ms/step
[60.50681948661804, 35.0]
1/1 [==============================] - 0s 32ms/step
[69.9232566356659, 35.0]
1/1 [==============================] - 0s 33ms/step
[67.70181059837341, 35.0]
1/1 [==============================] - 0s 35ms/step
[112.34268188476562, 35.0]
1/1 [==============================] - 0s 31ms/step
[106.4549732208252, 35.0]
1/1 [==============================] - 0s 31ms/step
[109.61449146270752, 35.0]
1/1 [==============================] - 0s 34ms/step
[83.08289051055908, 0.0]
1/1 [==============================] - 0s 33ms/step
[97.79930114746094, 0.0]
1/1 [==============================] - 0s 31ms/step
[88.12336444854736, 0.0]
1/1 [==============================] - 0s 35ms/step
[106.79656982421875, 0.0]
1/1 [==============================] - 0s 34ms/step
[86.5446829795

In [30]:
print(predictions[:5])

[[0.50756955, 0.0], [0.69492394, 1.0], [0.13133524, 1.0], [0.24904071, 1.0], [0.22127263, 1.0]]


In [31]:
df_pred = pd.DataFrame(predictions, columns=['angle', 'speed'])

# Add an 'image_id' column as the first column, set to be the index + 1
df_pred['image_id'] = np.arange(1, len(predictions) + 1)

# Reorder the DataFrame to have 'image_id' as the first column
df_pred = df_pred[['image_id', 'angle', 'speed']]

# Save the DataFrame to a CSV file, without including the index
df_pred.to_csv('submission_2.csv', index=False)